In [2]:
import time
import numpy as np
from tqdm import tqdm
from util.data_generate import *
from util.methods import *
import pandas as pd

# 实验参数
n = 1  # 实验重复次数
J = 10000  # 抽样次数
J1 = 100000 # 计算结果抽样次数
p, m = 50, 8  # 问题规模参数

mu = np.loadtxt('Data_generate/mu.txt')
Sigma = np.loadtxt('Data_generate/group_cov.txt')
std = np.sqrt(np.diag(Sigma))
R = Sigma / np.outer(std, std)

# 定义6种算法及其对应的索引获取函数
methods = [
    ('smallest', lambda mu: np.argsort(mu)[:m]),  # 直接取最小值
    ('cluster', lambda mu: find_min_mu_in_clusters(mu, cluster_subjects(R, n_clusters=m, method='average'))),  # 基于层次聚类的方法
    ('improvement', lambda mu: improvement(p, m, mu, std, R, Sigma)[1]),  # 基于集合贡献值的方法
    ('SAA', lambda mu: SAA(p, m, mu, std, R, Sigma, J=J)[1]),  # SAA
    ('CTO_mult', lambda mu: cyclic_multstart(p, m, mu, std, R, Sigma, J=J)[1])
]

rankings = np.zeros(5)  # 存储每种方法获胜次数
times = np.zeros(5)    # 存储每种方法总耗时

# 初始化输出文件
with open('output.csv', 'w') as f:
    f.write(','.join([m[0] for m in methods]) + '\n')

for seed in tqdm(range(n)):
    # 生成数据
    np.random.seed(0)
    
    # 运行所有方法
    B_list, indices_list = [], []
    for i, (name, get_indices) in enumerate(methods):
        start = time.perf_counter()
        indices = get_indices(mu)  # 获取选中的索引
        B = np.linalg.cholesky(Sigma[np.ix_(indices, indices)])  # Cholesky分解
        times[i] += time.perf_counter() - start  # 累计时间
        
        B_list.append(B)
        indices_list.append(indices)
    
    # 模拟计算各方法表现
    G = np.zeros(5)
    z_samples = np.random.randn(J*10, m)
    for i in range(5):
        G[i] += np.mean(np.min(mu[indices_list[i]] + (B_list[i] @ z_samples.T).T, axis=1))

    # 计算本次实验的名次
    ranks = np.argsort(np.argsort(G)) + 1
    rankings += ranks
    
    # 记录结果
    with open('output.csv', 'a') as f:
        f.write(','.join([f"{x:.6f}" for x in G]) + '\n')

# 记录并输出结果
data = pd.read_csv('output.csv')
smallest = np.mean(data['smallest'].to_numpy())
cluster = np.mean(data['cluster'].to_numpy())
improvement = np.mean(data['improvement'].to_numpy())
SAA = np.mean(data['SAA'].to_numpy())
cyclic = np.mean(data['CTO_mult'].to_numpy())
results = [smallest,cluster,improvement,SAA,cyclic]
avg_ranks = rankings / n

result_df = pd.DataFrame({
    '算法名称': [m[0] for m in methods],
    '平均名次': avg_ranks.round(2),
    '平均优化结果': results,
    '平均用时(秒)': times / n
})
result_df.to_csv('algorithm_result.csv', float_format='%.6f',index=False)
result_df

100%|██████████| 1/1 [00:07<00:00,  7.95s/it]


,算法名称,平均名次,平均优化结果,平均用时(秒)
0,smallest,3.0,51.708969,0.000066
1,cluster,4.0,52.249643,0.000408
2,improvement,2.0,51.690740,0.094798
3,SAA,1.0,50.885581,7.512958
4,CTO_mult,5.0,54.284236,0.328537
